This notebook is done following 
* [Building text classifier with Differential Privacy](https://github.com/pytorch/opacus/blob/main/tutorials/building_text_classifier.ipynb)
* [Fine-tuning with custom datasets](https://huggingface.co/transformers/v3.4.0/custom_datasets.html#seq-imdb)

# Intial Setup
https://huggingface.co/docs/transformers/training

## Install

In [1]:
!pip install opacus
# !pip install transformers
!pip install datasets
import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 KB 1.2 MB/s eta 0:00:00


## Import

In [2]:
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
import torch
from torch.utils.data import DataLoader

from opacus.utils.batch_memory_manager import BatchMemoryManager

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import gc

pd.set_option('display.max_columns', None)

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tokenize-ucberkeley-distilbert/__results__.html
/kaggle/input/tokenize-ucberkeley-distilbert/validation.pkl
/kaggle/input/tokenize-ucberkeley-distilbert/train.pkl
/kaggle/input/tokenize-ucberkeley-distilbert/test.pkl
/kaggle/input/tokenize-ucberkeley-distilbert/validation.csv
/kaggle/input/tokenize-ucberkeley-distilbert/__notebook__.ipynb
/kaggle/input/tokenize-ucberkeley-distilbert/__output__.json
/kaggle/input/tokenize-ucberkeley-distilbert/train.csv
/kaggle/input/tokenize-ucberkeley-distilbert/test.csv
/kaggle/input/tokenize-ucberkeley-distilbert/custom.css


## Config

In [4]:
from dataclasses import dataclass

@dataclass
class Config:
    model_name = 'distilbert-base-uncased'
    dataset_name = 'ucberkeley-dlab/measuring-hate-speech'
    tokenizer_root = '/kaggle/input/tokenize-ucberkeley-using-distilbert/'
    text_column = 'text'
    # if the raw id column is string, replace that with an integer index during preprocessing
    id_column = 'comment_id'

    # target in raw dataset. However, it will be renamed to `labels` here to facilitate training setup
    raw_target_column = 'hatespeech'
    target_column = 'labels'
    
    # If needs to be splitted into train test validation set
    need_to_split = True
    # if need_to_split is True, test and validation data with each be 50% of this amount
    test_size = 0.3
    max_seq_length = 128
    seed = 2022
    
    batch_size = 64
    learning_rate = 1e-3
    epochs = 10
    num_labels = 2
    
    # Private training config
    delta_list = [5e-2, 1e-3, 1e-6]
    noise_multiplier = 0.45
    max_grad_norm = 1
    max_physical_batch_size = 32
    target_epsilon = 9.0

## Set seed

In [5]:
import random

def seed_torch(seed=7):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    

global_seed = Config.seed
seed_torch(global_seed)

## Get device

In [6]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


# Load tokenized data

In [7]:
text = Config.text_column
target = Config.target_column

In [8]:
import pickle
    
with open(os.path.join(Config.tokenizer_root, 'train.pkl'), 'rb') as input_file:
    train_tokenized = pickle.load(input_file)
    input_file.close()
    
with open(os.path.join(Config.tokenizer_root, 'validation.pkl'), 'rb') as input_file:
    validation_tokenized = pickle.load(input_file)
    input_file.close()
    
with open(os.path.join(Config.tokenizer_root, 'test.pkl'), 'rb') as input_file:
    test_tokenized = pickle.load(input_file)
    input_file.close()

In [9]:
print(train_tokenized)

Dataset({
    features: ['comment_id', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 26994
})


# Private Training

## Data loader

[How to choose batch size in DP](https://github.com/pytorch/opacus/blob/main/tutorials/building_text_classifier.ipynb)

In [10]:
BATCH_SIZE = Config.batch_size
id_column = Config.id_column

train_dataloader = DataLoader(train_tokenized.remove_columns(id_column), batch_size=BATCH_SIZE)
validation_dataloader = DataLoader(validation_tokenized.remove_columns(id_column), batch_size=BATCH_SIZE*5)
test_dataloader = DataLoader(test_tokenized.remove_columns(id_column), batch_size=BATCH_SIZE*5)

## Model and optimizer

In [11]:
# add the utility script from File->Add utility script
from train_utils import TrainUtil, ModelCheckPoint, EarlyStopping

num_labels = Config.num_labels
model_name = Config.model_name
train_util = TrainUtil(Config.id_column, Config.target_column, device)

In [12]:
# load a fresh model each time
model = TrainUtil.load_pretrained_model(model_name, num_labels)

# Set the model to train mode (HuggingFace models load in eval mode)
model = model.train().to(device)
LEARNING_RATE = Config.learning_rate
# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)
EPOCHS = Config.epochs

# https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, verbose=True)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

Total parameters count: 66955010
Trainable parameters count: 7680002, percent 11.470


In [13]:
result_dir = ''
best_model_path = os.path.join(result_dir, 'model.pt')

if result_dir != '':
    os.makedirs(result_dir, exist_ok=True)

check_point = ModelCheckPoint(filepath=best_model_path)
early_stopping = EarlyStopping(patience=3, min_delta=0)

## Privacy Engine

In [14]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()

In [15]:
# model, optimizer, train_dataloader = privacy_engine.make_private(
#     module=model,
#     optimizer=optimizer,
#     data_loader=train_dataloader,
#     noise_multiplier=Config.noise_multiplier,
#     max_grad_norm=Config.max_grad_norm,
#     poisson_sampling=False,
# )

model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    target_delta=Config.delta_list[-1],
    target_epsilon=Config.target_epsilon, 
    epochs=EPOCHS,
    max_grad_norm=Config.max_grad_norm,
)

## Loop

In [16]:
start_epoch = 1
# load a previous model if there is any
# model, optimizer, lr_scheduler, start_epoch = load_model(model, optimizer, lr_scheduler, device, filepath=best_model_path)

for epoch in range(start_epoch, EPOCHS+1):
    gc.collect()
    
    with BatchMemoryManager(
        data_loader=train_dataloader, 
        max_physical_batch_size=Config.max_physical_batch_size, 
        optimizer=optimizer
    ) as memory_safe_data_loader:
        train_loss, train_result, train_probs = train_util.dp_train(
            model, optimizer, epoch, memory_safe_data_loader
        )
    val_loss, val_result, val_probs = train_util.evaluate(
        model, validation_dataloader, epoch, 'Validation'
    )

    epsilons = []
    for delta in Config.delta_list:
        epsilons.append(privacy_engine.get_epsilon(delta))

    print(
      f"Epoch: {epoch} | "
      f"ɛ: {np.round(epsilons, 2)} |"
      f"Train loss: {train_loss:.3f} | "
      f"Train result: {train_result} |\n"
      f"Validation loss: {val_loss:.3f} | "
      f"Validation result: {val_result} | "
    )
    
    loss = -val_result['f1']
    lr_scheduler.step(loss)
    check_point(model, optimizer, lr_scheduler, epoch, loss)
    
    early_stopping(loss)
    if early_stopping.early_stop:
        break
    print()

Epoch 1 (Train):   0%|          | 0/841 [00:00<?, ?it/s]

Epoch 1 (Validation):   0%|          | 0/19 [00:00<?, ?it/s]

Epoch: 1 | ɛ: [0.87 2.78 5.85] |Train loss: 1.031 | Train result: {'accuracy': 0.7157, 'f1': 0.001, 'auc': 0.683} |
Validation loss: 0.990 | Validation result: {'accuracy': 0.7179, 'f1': 0.0169, 'auc': 0.7828} | 

Loss improved from inf to -0.017. Saving model.



Epoch 2 (Train):   0%|          | 0/841 [00:00<?, ?it/s]

Epoch 2 (Validation):   0%|          | 0/19 [00:00<?, ?it/s]

...

Epoch 8 (Validation):   0%|          | 0/19 [00:00<?, ?it/s]

Epoch: 8 | ɛ: [2.1  4.63 8.51] |Train loss: 0.964 | Train result: {'accuracy': 0.7922, 'f1': 0.5711, 'auc': 0.8007} |
Validation loss: 1.136 | Validation result: {'accuracy': 0.7955, 'f1': 0.5617, 'auc': 0.8108} | 
Epoch 00008: reducing learning rate of group 0 to 1.0000e-04.
Early stopping counter 2 of 3



Epoch 9 (Train):   0%|          | 0/841 [00:00<?, ?it/s]

Epoch 9 (Validation):   0%|          | 0/19 [00:00<?, ?it/s]

Epoch: 9 | ɛ: [2.23 4.83 8.77] |Train loss: 0.932 | Train result: {'accuracy': 0.7955, 'f1': 0.5869, 'auc': 0.8111} |
Validation loss: 1.086 | Validation result: {'accuracy': 0.7971, 'f1': 0.5852, 'auc': 0.8153} | 
Early stopping counter 3 of 3
Early stopping..


In [17]:
# load the best model
model, _, _, best_epoch = TrainUtil.load_model(model, optimizer, lr_scheduler, device, filepath=best_model_path)

# make_private_with_epsilon function creates inconsistent train dataloader size
train_dataloader = DataLoader(train_tokenized.remove_columns(id_column), batch_size=BATCH_SIZE*5)
train_loss, train_result, train_probs = train_util.evaluate(model, train_dataloader, best_epoch, 'Train')
# no need to reevaluate if the validation set if the last model is the best one
if best_epoch != epoch:
    val_loss, val_result, val_probs = train_util.evaluate(model, validation_dataloader, best_epoch, 'Validation')
test_loss, test_result, test_probs = train_util.evaluate(model, test_dataloader, best_epoch, 'Test')

Loaded best model from epoch 6


Epoch 6 (Train):   0%|          | 0/85 [00:00<?, ?it/s]

Epoch 6 (Validation):   0%|          | 0/19 [00:00<?, ?it/s]

Epoch 6 (Test):   0%|          | 0/19 [00:00<?, ?it/s]

## Dump results and others

In [18]:
# Save the results
train_util.dump_results(
    result_dir,train_probs, train_tokenized, 
    val_probs, validation_tokenized, test_probs, test_tokenized
)

## Save config

In [19]:
import json

config_dict = dict(Config.__dict__)
# exclude hidden variables
keys = list(config_dict.keys())
for key in keys:
    if key.startswith('__'):
        del config_dict[key]
        
with open(os.path.join(result_dir, 'config.json'), 'w') as output:
    json.dump(config_dict, output, indent=4)